In [ ]:
import torch
import torch.nn as nn
import torchvision 
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim
import copy
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
device = 'cuda'

In [ ]:
imsize = 512
transform = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])

In [ ]:
def preprocess(img_path):
    img = Image.open(img_path)
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device, torch.float)
    return img   

In [ ]:
style_image_path = './picasso.jpg'
content_image_path = './dancing.jpg'

In [ ]:
style_image = preprocess(style_image_path)
content_image = preprocess(content_image_path)

In [ ]:
print(style_image.size())
print(content_image.size())

In [ ]:
unloader = transforms.ToPILImage()

In [ ]:
def show(img, title = None):
    temp = img.cpu().clone()
    temp = temp.squeeze(0)
    temp = unloader(temp)
    plt.title(title)
    plt.imshow(temp)
    

In [ ]:
show(content_image, 'content_image')

In [ ]:
show(style_image, 'style_image')

In [ ]:
def content_loss(a,b):
    return torch.mean((a-b)**2)

In [ ]:
def gram_matrix(inp):
    _,c,h,w = inp.size()
    temp = inp.view(c, h*w)
    G = torch.mm(temp, temp.t())
    return G/ (c * h * w)

In [ ]:
def style_loss(a,b):
    A = gram_matrix(a)
    B = gram_matrix(b)
    return torch.mean((A-B)**2)

In [ ]:
cnn = models.vgg19(pretrained = True).features

In [ ]:
mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
std = torch.tensor([0.229, 0.224, 0.225]).to(device)

In [ ]:
def normalize(image, mean, std):
    mean = mean.view(-1,1,1)
    std = std.view(-1,1,1)
    return (image - mean)/ std

In [ ]:
def denormalize(image, mean, std):
    mean = mean.view(-1,1,1)
    std = std.view(-1,1,1)
    return (image + mean) * std

In [ ]:
layers = {'0': 'conv1',
          '5': 'conv2', 
          '10': 'conv3', 
          '19': 'conv4',  ## content representation
          '28': 'conv5'}

In [ ]:
def get_layer_values(image, model, layers):
    temp = image
    layer_values = {}
    for name, layer in model._modules.items():
        name = name
        layer = layer.to(device)
        temp = layer(temp)
        if name in layers:
            layer_values[layers[name]] = temp
    return layer_values
    

In [ ]:
target_image = torch.randn(content_image.data.size(), device=device).requires_grad_(True)

In [ ]:
n_style_image = normalize(style_image, mean, std) 
n_content_image = normalize(content_image, mean ,std)

In [ ]:
content_layer_values = get_layer_values(n_content_image, cnn, layers)
style_layer_values = get_layer_values(n_style_image, cnn, layers)

In [ ]:
alpha = 1 #content weight
beta = 10000000 #style weight

In [ ]:
optimizer = optim.Adam([target_image], lr = 0.04)
epochs = 5000
show_every = 100

In [ ]:
show(target_image)

In [ ]:
for i in range(epochs):
    target_layer_values = get_layer_values(target_image, cnn, layers)
    closs = content_loss(content_layer_values['conv4'], target_layer_values['conv4'])
    sloss = 0
    for layer in layers.values():
        sloss += style_loss(style_layer_values[layer], target_layer_values[layer])
    total_loss = (alpha * closs) + (beta * sloss)
    print(f'total_loss : {total_loss}')
    print(f'sloss : {sloss}, closs : {closs}')
    #update the image
    optimizer.zero_grad()
    total_loss.backward(retain_graph= True)
    optimizer.step()
    

In [27]:
temp = target_image

NameError: name 'n_target_image' is not defined